## Combining the predictions of both the models

### Based on performance , the models are assigned weights

## Method1: Weights based Combining of results

In [2]:
import numpy as np

arima_predictions= np.load("first_30_pred_model1.npy")
ridge_predictions= np.load("first_30_pred_model2.npy")

# RMSE of both models
arima_rmse = 0.01701  
ridge_rmse = 1.4770   

# inverse of RMSE 
inverse_arima_rmse = 1 / arima_rmse
inverse_ridge_rmse = 1 / ridge_rmse

total_inverse_rmse = inverse_arima_rmse + inverse_ridge_rmse

# Weights
w_arima = inverse_arima_rmse / total_inverse_rmse  # Weight for ARIMA
w_ridge = inverse_ridge_rmse / total_inverse_rmse  # Weight for Ridge

combined_predictions = w_arima * arima_predictions + w_ridge * ridge_predictions
print("Combined Predictions:", combined_predictions)


Combined Predictions: [170.06182806 171.05630178 172.22170355 173.07001718 174.03869047
 175.19631227 176.03699866 177.11929791 178.32952041 178.96959931
 179.99067865 181.24787734 181.98326043 183.13280273 183.91873359
 184.95147515 186.10679445 186.88447313 187.92073091 188.873781
 189.87542547 190.84949506 191.83248812 192.82229514 193.82434752
 195.02184262 195.80944535 196.82495752 197.77922415 198.77641056]


### Inverse of RMSE is assigned as weight, so that the model with high error(RMSE) is assigned lesser weightage comparitively

## Evaluating combined results

In [4]:
actual_stockprice= np.load("actual_stockprice.npy")
print(actual_stockprice)

[176.69046021 396.81060791 225.6000061  175.6193339  415.527771
 216.79999542 162.72999573 265.27999878 273.57998657 171.11000061
 135.39472961 217.80000305 163.6700058  195.59666951 135.52000427
 186.2828064  230.16999817 174.40107727  99.14500046 166.2049942
 183.13000488 187.07000732 194.75588989 149.92999268 301.01934814
 244.62000275 308.03704834 412.29473877 181.54908752 148.66000366]


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = np.column_stack((arima_predictions, ridge_predictions))
y = actual_stockprice 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

meta_model = LinearRegression()
meta_model.fit(X_train, y_train)


combined_predictions = meta_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, combined_predictions))
print("Combined Predictions (Stacking):", combined_predictions)
print("RMSE (Stacking):", rmse)


Combined Predictions (Stacking): [190.45661144 211.48186721 196.79627114 207.29277727 228.28470119
 221.32126515]
RMSE (Stacking): 98.063154271082


### Optimizing the weights to combine the results more accurately

In [14]:
from scipy.optimize import minimize
import numpy as np

def objective(weights):
    w_arima, w_ridge = weights
    combined = w_arima * arima_predictions + w_ridge * ridge_predictions
    rmse = np.sqrt(np.mean((actual_stockprice - combined) ** 2))
    return rmse

constraints = {'type': 'eq', 'fun': lambda weights: 1 - sum(weights)}
bounds = [(0, 1), (0, 1)]

# Initializing weights
initial_weights = [0.5, 0.5]

result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)

# Optimal weights
w_arima, w_ridge = result.x
combined_predictions = w_arima * arima_predictions + w_ridge * ridge_predictions

# Calculate RMSE for the combined predictions
combined_rmse = np.sqrt(np.mean((actual_stockprice - combined_predictions) ** 2))
print(f"Optimal Weights: ARIMA={w_arima:.4f}, Ridge={w_ridge:.4f}")
print("Combined Predictions (Optimal Weights):", combined_predictions)
print(f"RMSE (Combined Predictions): {combined_rmse:.4f}")



Optimal Weights: ARIMA=0.0000, Ridge=1.0000
Combined Predictions (Optimal Weights): [179.68895393 180.12026157 195.56439726 183.15827764 181.3235002
 196.08431208 183.00828194 191.15341744 210.5342161  179.83857391
 182.60668737 206.11346357 183.78851054 197.83968916 179.95441114
 183.74683265 198.30541249 179.69533452 183.79658922 180.58960689
 181.65073008 180.28991734 179.7128636  179.73428888 180.83123936
 199.09417359 181.35573791 183.63487928 180.5347466  181.20431329]
RMSE (Combined Predictions): 84.3419


In [18]:
# Calculating RMSE for each model
arima_rmse = np.sqrt(np.mean((actual_stockprice - arima_predictions) ** 2))
ridge_rmse = np.sqrt(np.mean((actual_stockprice - ridge_predictions) ** 2))

print("Root Mean Squared Errors(RMSE) :")
print(f"RMSE (ARIMA Model): {arima_rmse:.4f}")

print(f"RMSE (Combined Predictions): {combined_rmse:.4f}")


Root Mean Squared Errors(RMSE) :
RMSE (ARIMA Model): 86.6996
RMSE (Combined Predictions): 84.3419


## The combined result is more accuarte than the model's predictions individually